# Types 

The **type** of a variable tells us the "shape" of the variable, i.e. how to treat / interpret the data stored in the block of memory associated with that variable.

Although it is possible to do much in Julia without thinking or worrying about types, they are always just under the surface, and the true power of the language becomes available through their use.

In [2]:
x = 3

3

In [3]:
typeof(x)

Int64

For basic ("primitive") types, we can see the raw bits that are associated to a variable:

In [5]:
bits(x)  # bitstring in Julia 0.7

"0000000000000000000000000000000000000000000000000000000000000011"

In [6]:
y = 3.0

3.0

In [7]:
bits(y)

"0100000000001000000000000000000000000000000000000000000000000000"

The internal representations are different, corresponding to the different types. 

We can treat the storage as being of a different type:

In [9]:
z = reinterpret(Int, y)

4613937818241073152

In [10]:
h = hex(z)

"4008000000000000"

In [11]:
z1 = parse(Int, h, 16)  # base 16

4613937818241073152

In [12]:
bits(z1)

"0100000000001000000000000000000000000000000000000000000000000000"

# Defining our own types

We have previously seen some examples of how to define a new type in Julia. 

A type definition can be thought of as template for a kind (type) of box, that contains certain **fields** containing data.

One of the simplest examples is a "volume" type, representing the volume of some physical or mathematical object:

In [13]:
struct Vol
    value
end

In [14]:
Vol

Vol

This defines a new type, called `Vol`, containing one field, called `value`. 

It does not yet create an object that has that type. That is done by calling a **constructor** -- a special function with the same name as the type:

In [16]:
V = Vol(3)

Vol(3)

In [17]:
V

Vol(3)

`V` is a Julia variable that is of type `Vol`:

In [18]:
typeof(V)

Vol

Its "shape" is thus that of a box containing itself a single variable, which we can access as

In [37]:
V.value

3

Since we defined the type `Vol` as `struct`, we cannot modify the contents of the object once it has been created:

In [20]:
V.value = 10

LoadError: [91mtype Vol is immutable[39m

We could instead make a mutable object:

In [21]:
mutable struct Vol1
    value
end

In [22]:
V = Vol1(3)

Vol1(3)

In [23]:
V.value

3

In [24]:
V.value = 10

10

In [25]:
V

Vol1(10)

We can change how our objects look by defining a new method of the `show` function:

In [26]:
import Base: show

show(io::IO, V::Vol1) = print(io, "Volume with value ", V.value)

show (generic function with 268 methods)

In [27]:
V = Vol1(3)

Volume with value 3

We can define e.g. the sum of two volumes:

In [28]:
import Base: +

+(V1::Vol1, V2::Vol1) = Vol1(V1.value + V2.value)

+ (generic function with 181 methods)

In [29]:
V + V

Volume with value 6

But the following does not work, since we haven't defined `*` yet on our type:

In [30]:
2V

LoadError: [91mMethodError: no method matching *(::Int64, ::Vol1)[0m
Closest candidates are:
  *(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:424
  *(::Real, [91m::Complex{Bool}[39m) at complex.jl:254
  *(::Real, [91m::Complex[39m) at complex.jl:266
  ...[39m

**Exercise**: Define `*` of two `Vol`s  and of a `Vol` and a number.

## Type annotations

There is a problem with our definition:

In [31]:
Vol1("hello")

Volume with value hello

It doesn't make sense to have a string as a volume. So we should **restrict** which kinds of `value` are allowed, by specifying ("annotating") the type of `value` in the type definition:

In [1]:
struct Vol2
    value::Float64
end

In [51]:
Vol2(3.1)

Vol2(3.1)

In [2]:
Vol2("hello")

LoadError: [91mMethodError: Cannot `convert` an object of type String to an object of type Float64
This may have arisen from a call to the constructor Float64(...),
since type constructors fall back to convert methods.[39m

# Parameterizing a type

In different contexts, we may want integer volumes, or rational volumes, rather than `Vol`s which contain a floating-point number, e.g. for a 3D printer that makes everything out of cubes of the same size. 

We could define the following sequence of different types.

In [3]:
type Vol_Int
    value::Int
end

type Vol_Float
    value::Float64
end

type Vol_Rational
    value::Rational{Int64}
end

In [4]:
Vol_Int(3)

Vol_Int(3)

In [5]:
Vol_Int(3.1)

LoadError: [91mInexactError()[39m

In [6]:
Vol_Float(3.1)

Vol_Float(3.1)

But clearly this is the wrong way to do it, since we're repeating ourselves, leading to inefficiency and buggy code. (https://en.wikipedia.org/wiki/Don't_repeat_yourself).

Can't Julia itself automatically generate all of these different types?

## Specifying type parameters

What we would like to do is tell Julia that the **type** itself (here, `Int`, `Float64` or `Rational{Int64}`) 
is a special kind of **parameter** that we will specify. 

To do so, we use curly braces (`{`, `}`) to specify such **type parameter** `T`:

In [1]:
struct Vol3{T}
    value
end

We can now create objects of type `Vol3`  with any type `T`:

In [2]:
V = Vol3{Float64}(3.1)

Vol3{Float64}(3.1)

In [60]:
typeof(V)

Vol3{Float64}

In [3]:
V2 = Vol3{Int64}(4)

Vol3{Int64}(4)

In [4]:
typeof(V2)

Vol3{Int64}

We see that the types of `V1` and `V2` are *different* (but related), and we have achieved what 
we wanted.

The type `Vol3` is called a **parametric type**, with **type parameter** `T`. Parameteric types may have several type parameters, as we have already seen with `Array`s:

In [66]:
a = [3, 4, 5]
typeof(a)

Array{Int64,1}

The type parameters here are `Int64`, which is itself a type, and the number `1`.

## Improving the solution

The problem with this solution is the following, which echos what happened at the start of the notebook:

In [5]:
V = Vol3{Int64}(3.1)

Vol3{Int64}(3.1)

In [6]:
typeof(V.value)

Float64

The type `Float64` of the field `V.value` is distinct from the type parameter `Int64` we specified. 
So we have not yet actually captured the pattern of `Vol2`,
which restricted the `value` field to be of the desired type.

We solve this be specifying the field to **also be of type `T`**, with the **same `T`**:

In [7]:
struct Vol4{T}
    value::T
end

For example,

In [8]:
V = Vol4{Int64}(3)

Vol4{Int64}(3)

If necessary and possible, the argument to the constructor will be converted to the parametric type `T` that we specify:

In [9]:
V = Vol4{Int64}(3.0)

Vol4{Int64}(3)

In [71]:
typeof(V.value)

Int64

Now when we try to do 

In [10]:
Vol4{Int64}(3.1)

LoadError: [91mInexactError()[39m

Julia throws an error, namely an `InexactError()`.
This means that we are trying to "force" the number 3.1 into a "smaller" type `Int64`, i.e. one in which it can't be represented.

However, now we seem to be repeating ourselves again: We know that `3.1` is of type `Float64`, and in fact Julia knows this too; so it seems redundant to have to specify it. Can't Julia just work it out? Indeed, it can!:

In [12]:
Vol4(3.1)

Vol4{Float64}(3.1)

Here, Julia has **inferred** the type `T` from the "inside out". That is, it performed pattern matching to realise that `value::T` is **matched** if `T` is chosen to be `Float64`, and then propagated this same value of `T` **upwards** to the type parameter.

## More fields

**Exercise**: Define a `Point` type that represents a point in 2D, with two fields. What are the options for this type, mirroring the types `Vol1` through `Vol4`?

## Summary

With parametric types, we have the following possibilities:

1. Julia converts (if possible) to the header type

2. Julia infers the header type from the inside (through the argument)


# Constructors

When we define a type, Julia also defines the **constructor functions** that we have been using above. These are functions with exactly the same name as the type.

They can be discovered using `methods`:

In [15]:
struct Vol1
    value
end

In [16]:
methods(Vol1)

# 2 methods for generic function "(::Type)":
Vol1(value) in Main at In[15]:2
(::Type{T})(arg) where T in Base at sysimg.jl:24

We see that Julia provides two default constructors. [Note that the output has changed in Julia 0.7.]

For parametric types, it is a bit more complicated:

In [17]:
methods(Vol4)

# 2 methods for generic function "(::Type)":
(::Type{Vol4})(value::T) where T in Main at In[7]:2
(::Type{T})(arg) where T in Base at sysimg.jl:24

In [18]:
methods(Vol4{Float64})

# 2 methods for generic function "(::Type)":
(::Type{Vol4{T}})(value) where T in Main at In[7]:2
(::Type{T})(arg) where T in Base at sysimg.jl:24

## Outer constructors

Julia allows us to provide our own constructor functions.
E.g.

In [2]:
struct Vol1
    value
end

In [7]:
struct Vol2
    value::Float64
end

In [9]:
Vol2(3)

Vol2(3.0)

In [10]:
Vol2("3.1")

LoadError: [91mMethodError: Cannot `convert` an object of type String to an object of type Float64
This may have arisen from a call to the constructor Float64(...),
since type constructors fall back to convert methods.[39m

Here, we have tried to provide a numeric string, which is not allowed, since the string is not a number. We can add a constructor to allow this:

In [11]:
Vol2(s::String) = Vol2(parse(Float64, s))

Vol2

In [12]:
Vol2("3.1")

Vol2(3.1)

In [14]:
Vol2("hello")

LoadError: [91mArgumentError: cannot parse "hello" as Float64[39m

We have added a new constructor outside the type definition, so it is called an **outer constructor**.

## Constructors that impose a restriction: **inner constructors**

Now consider the following:

In [15]:
Vol4(3)

Vol4{Int64}(3)

In [16]:
V = Vol4(3.5)

Vol4{Float64}(3.5)

In [18]:
V.value

3.5

In [19]:
Vol4(-1)

Vol4{Int64}(-1)

Oops! A volume cannot be negative, but the constructors so far have no restrictions, and so allow us to make a negative volume. To prevent this, Julia allows us to provide our own constructor, in which any restrictions are enforced.

These constructors are written **within the type definition itself**, and hence are called **inner constructors**.

[In Julia, these are the **only methods** that may be defined inside the type definition. Unlike in object-oriented languages, methods **do not belong to types** in Julia; rather, they exist outside any particular type, and (multiple) dispatch is used instead.]

For example:

In [1]:
struct Vol5
    value::Float64
    
    function Vol5(V) 
        if V < 0
            throw(ArgumentError("Volumes cannot be negative"))
        end
        
        new(V)
    end
end

In [2]:
Vol5(3)

Vol5(3.0)

In [3]:
Vol5(-34)

LoadError: [91mArgumentError: Volumes cannot be negative[39m

If we define an inner constructor, then Julia no longer defines the standard constructors; this is why defining an inner constructor gives us exclusive control over how our objects are created.

Note that we use a special function `new` to actually create the object by filling in the values of the fields.

If we use an immutable object (defined using `struct`), there is no way of changing the value of the field stored inside the object, so the invariant that `value` must be positive can never be violated.

# Parametric functions

Since we now have the ability to make parametric types, we may wish to define parametric functions on those types. E.g.

In [9]:
struct Length{T}
    length::T
end

In [10]:
l = Length(10)

Length{Int64}(10)

In [19]:
function square_area(l)
    return l^2
end

square_area (generic function with 3 methods)

Suppose that we wish to round the area of squares with floating-point side length:

In [20]:
function square_area(l::T) where {T <: AbstractFloat}  # method for types T that are subtypes of AbstractFloat
    return ceil(Int, l^2)
end

square_area (generic function with 4 methods)

In [21]:
square_area(11.1)

124

# Inner constructors for parametric types

In [2]:
struct Vol6{T<:Real}
    value::T
    
    function Vol6{T}(V) where {T<:Real}   # where specifies that T is a parameter of the parametric function Vol6
        if V < 0
            throw(ArgumentError("Negative"))
        end

        return new{T}(V)
    end
end

Here, we have used the syntax for parametric functions to specify a parametric inner constructor

In [3]:
Vol6(3)

LoadError: [91mMethodError: Cannot `convert` an object of type Int64 to an object of type Vol6
This may have arisen from a call to the constructor Vol6(...),
since type constructors fall back to convert methods.[39m

In [4]:
Vol6{Float64}(3.1)

Vol6{Float64}(3.1)

We see that so far, we must explicitly specify the parametric type.

In [7]:
methods(Vol6)

# 1 method for generic function "(::Type)":
(::Type{T})(arg) where T in Base at sysimg.jl:24

We can again make Julia infer the type for us:

In [8]:
Vol6(x::T) where {T<:Real} = Vol6{T}(x)

Vol6

In [9]:
Vol6(3.1)

Vol6{Float64}(3.1)

In [10]:
Vol6(3)

Vol6{Int64}(3)

In [11]:
methods(Vol6)

# 2 methods for generic function "(::Type)":
(::Type{Vol6})(x::T) where T<:Real in Main at In[8]:1
(::Type{T})(arg) where T in Base at sysimg.jl:24

In [13]:
x = 3//4  # rational number

3//4

In [17]:
Vol6(x)

Vol6{Rational{Int64}}(3//4)

# Fixed-size objects are efficient

In [20]:
struct Vec{T}
    x::T
    y::T
end

How efficient is this?

In [21]:
import Base: +

+(f::Vec{T}, g::Vec{T}) where {T} = Vec(f.x + g.x, f.y + g.y)

+ (generic function with 181 methods)

In [22]:
using BenchmarkTools

INFO: Recompiling stale cache file /Users/dpsanders/.julia/lib/v0.6/HDF5.ji for module HDF5.
INFO: Recompiling stale cache file /Users/dpsanders/.julia/lib/v0.6/JLD.ji for module JLD.


In [23]:
@btime +(Vec(1.0, 2.0), Vec(1.0, 2.0))

  1.648 ns (0 allocations: 0 bytes)


Vec{Float64}(2.0, 4.0)

In [24]:
@btime [1.0, 2.0] + [1.0, 2.0]

  98.702 ns (3 allocations: 288 bytes)


2-element Array{Float64,1}:
 2.0
 4.0

Using fixed-size objects is much more efficient (50 times more efficient)! They are defined in the `StaticArrays.jl` package.